Online SPECULATION on intagram rate limiting
* 60 actions an hour
* max 200 follows in 24 hours
* Per hour: Most sources suggest a limit around 60-100 follows per hour to avoid triggering Instagram's abuse detection. Going above this increases the risk of being flagged.
* Age of your account
In accordance with Instagram policy, every profile that was created more than three months ago is considered as old. Like, comments, follow/unfollow limits set for such accounts are higher
* Besides, you are recommended to subscribe to no more than 10 users per hour. 
* If your page is comparatively new, the follow and unfollow limit can be reduced to 150 Instagram users every day.
* Try to increase it gradually, starting from 50 accounts per day on the first week, 100 on the second, and 150 on the third.



In [ ]:
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from dotenv import load_dotenv
import pandas as pd
import random
import time
import json
import os
load_dotenv()



In [2]:
# Usernames to follow
df = pd.read_csv('data/third_batch_cleaned.csv')

# Record of accounts already attempted (as not to bother uninterested accounts)
tried2 = pd.read_csv("data/usernames_tried2.csv")

In [3]:
# Counter for accounts followed to not hit rate limit
FOLLOW_COUNT_FILE = 'data/followed_count.json'

with open(FOLLOW_COUNT_FILE, 'w') as f:
        json.dump({'count': 0}, f)           

# Position on usernames list to pick up from 
FOLLOW_INDEX_FILE = 'data/folowed_index.json'

In [6]:
# Hidden account login details
account_username = os.getenv("IG_USERNAME")
account_password = os.getenv("IG_PASSWORD")


In [7]:
# Rejecting cookies request pop-up on login 
def decline_cookies(driver):
    try:
        decline_cookies_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[text()='Decline optional cookies']"))
        )
        time.sleep(random.uniform(0.5,2))
        decline_cookies_button.click()
        print(f"declined_cookies")
    except Exception as e:
        print(f"Could not decline_cookies")

# Boolean function whether an element is present on the page
def is_element_present(driver, xpath, timeout=3):
    try:
        WebDriverWait(driver, timeout).until(EC.presence_of_element_located((By.XPATH, xpath)))
        return True
    except TimeoutException:
        return False

# Clicking the follow button on an instagram account page
def follow_user(driver, username):
    try:
        follow_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//div[text()='Follow']"))
        )
        time.sleep(random.uniform(0.5,2))
        follow_button.click()
        print(f"Followed {username}")
    except Exception as e:
        print(f"Could not follow {username}: {e}")

# Clicking the follow button on an instagram account page specifically when they have already followed you 
def follow_back_user(driver,username):
    try:
        follow_back_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//div[text()='Follow Back']"))
        )
        time.sleep(random.uniform(0.5,2))
        follow_back_button.click()
        print(f"Followed back {username}")
    except Exception as e:
        print(f"Could not follow back {username}: {e}")

# Standardising follower count for an instagram account
def convert_follower_count(follower_str):
    follower_str = follower_str.replace(',', '').strip()
    if 'K' in follower_str:
        return int(float(follower_str.replace('K', '')) * 1000)
    if 'M' in follower_str:
        return int(float(follower_str.replace('M', '')) * 1000000)
    return int(follower_str)

# Finding the number of account followers
def get_follower_count(driver):
    try:
        follower_element = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, "//ul/li[2]//span"))
        )
        followers_text = follower_element.text  
        followers = convert_follower_count(followers_text)  
        return followers
    except Exception as e:
        print(f"Error getting follower count: {e}")
        return 0  
    
# Random scrolling on page to reduce instagram bot detection
def random_scroll(driver,max):
    
    for i in range(random.randint(1,max + 1)):
        scroll_distance = random.randint(200, 500)  
        driver.execute_script(f"window.scrollBy(0, {scroll_distance});")
        time.sleep(random.uniform(0.3,2))
        
        """if (random.random() > 0.5):
            random_click(driver)
            time.sleep(random.uniform(0.3,1))"""
        
        if (random.random() > 0.75):
            scroll_distance = -random.randint(200, 300) 
            driver.execute_script(f"window.scrollBy(0, {scroll_distance});")
            time.sleep(random.uniform(0.3,2))
        
        if (random.random() > 0.9):
            scroll_distance = -random.randint(50, 120)  
            driver.execute_script(f"window.scrollBy( {scroll_distance},0);")
            time.sleep(random.uniform(0.3,1))
            
        time.sleep(random.uniform(0.3, 0.5))

    rand_scroll_up_dist = random.randint(20, 100)
    driver.execute_script("window.scrollBy(0, -(window.pageYOffset + arguments[0]));", rand_scroll_up_dist)
    time.sleep(random.uniform(1,2))
    
# Random clicking on page to reduce instagram bot detection
# Outisde of pictures borders as to not disturb automatic process by redirecting to another page
def random_click(driver):
    try:
        width = driver.execute_script("return window.innerWidth;")
        height = driver.execute_script("return window.innerHeight;")

        x = random.randint(100, width - 100)  
        y = random.randint(200, height - 100)  

        driver.execute_script(f"document.elementFromPoint({x}, {y}).click();")
        print(f"Randomly clicked at coordinates ({x}, {y}).")
    except Exception as e:
        print(f"Error during random click: {e}")

# Saving a list of the usernames already attempted to follow
def save_tried2(tried2):
    tried2.to_csv('data/usernames_tried2.csv',index=False)


In [8]:
# Saving login cookies
def save_cookies(driver, path="cookies.json"):
    with open(path, 'w') as file:
        json.dump(driver.get_cookies(), file)

# Load cookies to refresh session
def load_cookies(driver, path="cookies.json"):
    if os.path.exists(path):
        with open(path, 'r') as file:
            cookies = json.load(file)
            for cookie in cookies:
                driver.add_cookie(cookie)
        return True
    return False

# Return number of accounts followed in session
def load_followed_count():
    if os.path.exists(FOLLOW_COUNT_FILE):
        with open(FOLLOW_COUNT_FILE, 'r') as f:
            return json.load(f).get('count', 0)
    return 0

# Saving the sessions attempted followed count
def save_followed_count():
    count = load_followed_count() + 1
    
    with open(FOLLOW_COUNT_FILE, 'w') as f:
        json.dump({'count': count}, f)

# Returns index of username list to resume following from        
def load_followed_index():
    if os.path.exists(FOLLOW_INDEX_FILE):
        with open(FOLLOW_INDEX_FILE, 'r') as f:
            return json.load(f).get('count', 0)
    return 0

# Saves index in usernames list of point reached
def save_followed_index():
    count = load_followed_index() + 1
    with open(FOLLOW_INDEX_FILE, 'w') as f:
        json.dump({'count': count}, f)


In [ ]:
# Open Chrome driver and direct to instagram home page

service = Service('./chromedriver')

driver = webdriver.Chrome(service=service)

driver.get("https://www.instagram.com")

time.sleep(random.randint(1,4))



# Log into account with saved cookies if there otherwise account details
if load_cookies(driver):
    print("Loaded cookies, trying to bypass login...")
    driver.refresh()  
    time.sleep(5)  
else:
    time.sleep(random.randint(2, 5))

    username_input = driver.find_element(By.NAME, 'username')
    password_input = driver.find_element(By.NAME, 'password')

    username_input.send_keys(account_username)
    time.sleep(random.randint(2, 5))
    password_input.send_keys(account_password)
    time.sleep(random.randint(1, 3))
    password_input.send_keys(Keys.RETURN)

    time.sleep(5)  
    save_cookies(driver)  

print("Login successful")

In [ ]:
followed_count = load_followed_count()
followed_index = load_followed_index()
print(f" index =  {followed_index}")


print(f"Starting with {followed_count} accounts followed today.")


In [ ]:
# Number of accounts to attempt to follow
ntf = 100
usernames = df.data[followed_index:followed_index + ntf]

In [ ]:

i = 0
time.sleep(random.uniform(2,4))

for username in usernames:
    save_followed_index()

    # Saves account usernames attempted every 5 iterations
    i += 1
    if i % 5 == 0:
        print(i)
        save_tried2(tried2)

    # Skip a username if it's already been attempted
    if username in list(tried2['Unnamed: 0']):
        print('Tried before',username)
        continue
    
    # Add username to the end of the attempted list
    tried2.loc[len(tried2)] = username
    
    # Direct to the usernames account page
    driver.get(f"https://www.instagram.com/{username}/")
    time.sleep(random.uniform(0.5, 1.3))
    
    
    # Skip username if there is no account linked to the username
    if is_element_present(driver, "//span[contains(text(), 'Sorry, this page')]"):
        print(f"Skipping {username}: The account does not exist.")
        time.sleep(random.uniform(1, 2))
        continue
    
    # Skip username if the account is already followed or has been attempted already
    if is_element_present(driver, "//div[@class='_ap3a _aaco _aacw _aad6 _aade' and (text()='Following' or text()='Requested')]"):
        print(f"Skipping {username}: The account is already followed.")
        time.sleep(random.uniform(1, 2))
        random_scroll(driver,1)
        continue
    
    followers = get_follower_count(driver)
    
    time.sleep(random.uniform(0.5, 2))

    # Whether to follow private accounts or not (and based on follower count)
    if not is_element_present(driver, "//span[contains(text(), 'This account is private')]"):
        random_scroll(driver,1)
        if followers < 90:
            print(f"Skipping {username}: Less than 90 followers.")
            time.sleep(random.uniform(1, 2))
            continue
    else:
        # If all else passed then randomly scroll on page before following
        random_scroll(driver,5)
        if followers < 150:
            print(f"Skipping {username}: Less than 150 followers.")
            time.sleep(random.uniform(1, 2))
            continue
    
    time.sleep(random.uniform(0.5, 1))


    if (random.random() > 0.97):
        print("skipped for unpredictability")
        continue
    
    
    follow_user(driver, username)
    
    save_followed_count()
    
    time.sleep(random.uniform(1, 5))

save_tried2(tried2)

print(f"finished with {load_followed_count()} accounts followed today.")

